In [300]:
import requests
from datetime import timedelta
import json
import pandas as pd
import plotly.express as px

url = "https://rootd4.vercel.app/score/all"
response = requests.get(url)
dados = response.json()
# Transformar a resposta JSON em DataFrame
df = pd.DataFrame(response.json()).copy()


In [301]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.width', 120)
pd.set_option('expand_frame_repr', True)

## Quantidade de partidas

In [302]:
df.shape[0]

165

## ID Pokemon

In [303]:
def support_id_nome(id):
    support_names = {
        0: "Arcanine",
        1: "Azumarill",
        2: "Flareon",
        3: "Kingler",
        4: "Leafeon",
        5: "Glalie",
        6: "Beedrill",
        7: "Vaporeon",
        8: "Electabuzz",
        9: "Ampharos",
        10: "Muk",
        11: "Jolteon",
        12: "Rhydon",
        13 : "MrMime",
        14: "Swampert",
        15: "Glaceon",
        16: "Alakazam",
        17: "Pidgeot",
        18: "Meganium",
        19: "Umbreon",
        20: "Victribell",
        21: "Espeon",
        22: "Magneton"
    }
    return support_names.get(int(id), "Unknown Support")


def hero_nome_id(id):
    hero_names = {
        0: "Bulbasaur",
        1: "Charmander",
        2: "Squirtle",
        3: "Pikachu"
    }
    return hero_names.get(int(id), "Unknown Hero")

In [304]:
df['info'] = df['info'].apply(json.loads)

In [305]:
# Normalizar 'info'
info_df = pd.json_normalize(df['info'].tolist())

# Adicionar 'name' em info
info_df['name'] = df['name']
info_df['time'] = pd.to_numeric(df['time'])

# Filtrar registros com pokemon diferente de -1
stats_df = pd.json_normalize(df['info'].tolist(), 'statistics')
stats_df['pokemon'] = stats_df['pokemon'].astype(int)

# Adicionar 'name' em stats
stats_df['name'] = df['name'].repeat(info_df['statistics'].apply(len).tolist()).reset_index(drop=True)

stats_dfo = stats_df
stats_df = stats_df.query('pokemon != -1')

## TOP 5 Suportes usados por names

In [306]:
top5_suportes_jogadores = info_df

stats_df = stats_df.query('pokemon != -1')

top5_suportes_jogadores = (
    stats_df.groupby(['name', 'pokemon'])
    .size()
    .reset_index(name='count')
    .sort_values(by=['name', 'count'], ascending=[True, False])
    .groupby('name', as_index=False)  # Evita a criação de índice adicional
    .head(5)
    .groupby('name')['pokemon']
    .agg(lambda x: ', '.join(map(str, x)))
    .reset_index(name='Top 5 Suportes')
)

# Adicionar nomes de Pokémon usando a função support_id_nome
top5_suportes_jogadores['Top 5 Suportes'] = top5_suportes_jogadores['Top 5 Suportes'].apply(lambda x: ', '.join(map(support_id_nome, x.split(','))))


# Exibir o DataFrame resultante
top5_suportes_jogadores[['name', 'Top 5 Suportes']]



,name,Top 5 Suportes
0,Delirio,"Arcanine, Muk, Jolteon, MrMime, Glalie"
1,Guiltybr,"Arcanine, Kingler, Glalie, Electabuzz, Muk"
2,Jihn,"Electabuzz, Umbreon, Magneton, Arcanine, Kingler"
3,Jumbrado,"Arcanine, Beedrill, Electabuzz, Kingler, Muk"
4,KNOCK,"Arcanine, Kingler, Ampharos, Muk, Jolteon"
5,Ks,"Arcanine, Meganium, Kingler, Beedrill, Muk"
6,Lipekill,"Arcanine, Magneton, Kingler, Electabuzz, Muk"
7,Lironi,"Arcanine, Electabuzz, Kingler, Glaceon, Meganium"
8,Luhmb,"Kingler, Beedrill, Electabuzz, Alakazam, Magneton"
9,MatthawsRdC,"Jolteon, MrMime, Espeon, Glalie, Vaporeon"


## Suportes mais utilizados

In [307]:
# Contar a ocorrência de cada Pokémon
suportes_mais_usados = stats_df['pokemon'].value_counts().reset_index(name='count')

# Adicionar nomes de Pokémon usando a função support_id_nome
suportes_mais_usados['pokemon'] = suportes_mais_usados['pokemon'].apply(support_id_nome)

# Criar o gráfico de barras com o Plotly
fig = px.bar(suportes_mais_usados, x='pokemon', y='count', title='Pokémons de Suporte Mais Usados')
fig.update_layout(xaxis=dict(tickangle=45))  # Ajustar a rotação dos rótulos para melhor legibilidade

# Exibir o gráfico
fig.show()

#suportes_mais_usados


## Maior tempo de jogo

In [308]:
#
aux_df = info_df

# Calcular a quantidade total de tempo jogado por 'name' em segundos
tempo_total_jogo_nome = aux_df.groupby('name')['time'].sum().reset_index(name='tempo_total_segundos')

# Converter os segundos para horas
tempo_total_jogo_nome['tempo_total_horas'] = tempo_total_jogo_nome['tempo_total_segundos'] / 3600

# Converter os segundos para formato de hora
tempo_total_jogo_nome['tempo_total_horas'] = pd.to_datetime(tempo_total_jogo_nome['tempo_total_segundos'], unit='s').dt.strftime('%H:%M:%S')

# Exibir a tabela
#tempo_total_jogo_nome[['name', 'tempo_total_horas']]

# Criar um gráfico de barras interativo com o Plotly
fig = px.bar(
    tempo_total_jogo_nome,
    x='name',
    y='tempo_total_horas',
    labels={'total_time_seconds': 'Tempo Total Jogado (segundos)'},
    title='Distribuição do Tempo Total Jogado por Jogador',
)

# Personalizar layout
fig.update_layout(
    xaxis_title='Jogador',
    yaxis_title='Tempo Total Jogado',
    xaxis=dict(tickangle=45, tickmode='array', tickvals=tempo_total_jogo_nome['name'], ticktext=tempo_total_jogo_nome['name']),
)

# Exibir o gráfico
fig.show()

## Jogadores Vitórias

In [309]:
# Criar uma tabela com a contagem de cada jogador
vitorias_jogadores = info_df.groupby('name').size().reset_index(name='vitorias').sort_values(by='vitorias', ascending=False)

# Converter a coluna 'time' para numérica
info_df['time'] = pd.to_numeric(info_df['time'], errors='coerce')

# Calcular a média de tempo para cada jogador em minutos
media_tempo = info_df.groupby('name')['time'].mean().reset_index(name='media_tempo')
media_tempo['media_tempo'] /= 60  # Convertendo para minutos
media_tempo['media_tempo'] = pd.to_datetime(media_tempo['media_tempo'], unit='m').dt.strftime('%M:%S.%f')

# Exibir a tabela
resultado_final = pd.merge(vitorias_jogadores, media_tempo, on='name')

resultado_final



,name,vitorias,media_tempo
0,Nico,30,23:34.903861
1,Xanad0,29,23:02.112898
2,kawa,21,23:47.180623
3,Royal,18,24:08.372226
4,NewJeans,9,24:06.641777
5,Jumbrado,8,24:10.686492
6,Weiss,7,27:30.258562
7,Ks,6,26:34.745015
8,Mestre LpK,6,37:07.591488
9,ftbr,5,26:19.986206


## Quantidade jogada heróis

In [310]:
# Criar um DataFrame com os dados
contagem_heroi = info_df

contagem_heroi['hero_nome'] = contagem_heroi['character'].apply(hero_nome_id)

# Criar uma tabela com a contagem de cada personagem usando groupby
contagem_heroi = info_df.groupby(['hero_nome']).size().reset_index(name='quantidade')

# Exibir o gráfico
#contagem_heroi

# Criar um gráfico de pizza com Plotly Express
fig = px.pie(contagem_heroi, values='quantidade', names='hero_nome', title='Contagem de Personagens por Nome de Herói')

fig.show()


## Herói mais jogado por jogador

In [311]:
heroi_preferido_por_jogador = info_df 

# Calcular a contagem de ocorrências de cada personagem por jogador
heroi_preferido_por_jogador = heroi_preferido_por_jogador.groupby(['name', 'hero_nome']).size().reset_index(name='quantidade')

# Encontrar o índice do primeiro valor máximo para cada jogador
idx = heroi_preferido_por_jogador.groupby('name')['quantidade'].idxmax()
heroi_preferido_por_jogador = heroi_preferido_por_jogador.loc[idx]

# Exibir a tabela final
#heroi_preferido_por_jogador[['name', 'hero_nome', 'quantidade']]


## Suporte mais jogado pelos jogadores

In [312]:
suporte_preferido_por_jogador = stats_df

# Adicionar uma coluna com os nomes dos Heróis usando a função
suporte_preferido_por_jogador['support_nome'] = suporte_preferido_por_jogador['pokemon'].apply(support_id_nome)

suporte_preferido_por_jogador = suporte_preferido_por_jogador.groupby(['name', 'support_nome']).size().reset_index(name='quantidade')

# Exibir as primeiras linhas do DataFrame normalizado
idx = suporte_preferido_por_jogador.groupby('name')['support_nome'].idxmax()
suporte_preferido_por_jogador = suporte_preferido_por_jogador.loc[idx]

suporte_preferido_por_jogador[['name', 'support_nome', 'quantidade']]

,name,support_nome,quantidade
5,Delirio,Muk,2
15,Guiltybr,Umbreon,1
22,Jihn,Umbreon,2
33,Jumbrado,Umbreon,1
45,KNOCK,Vaporeon,1
59,Ks,Victribell,1
67,Lipekill,Victribell,1
79,Lironi,Swampert,1
84,Luhmb,Magneton,1
91,MatthawsRdC,Victribell,1


# DPS

## Geral

In [386]:
dps_geral = stats_dfo.copy()

dps_geral['nome'] = dps_geral['pokemon'].apply(support_id_nome)

# Adicionar colunas 'name' e 'time' de volta
dps_geral['time'] = df['time'].repeat(info_df['statistics'].apply(len).tolist()).reset_index(drop=True)

# Converter colunas para float
dps_geral['startTime'] = dps_geral['startTime'].str.replace(',', '.').astype(float)
dps_geral['time'] = dps_geral['time'].str.replace(',', '.').astype(float)
dps_geral['damage'] = dps_geral['damage'].str.replace(',', '.').astype(int)

# Calcular DPS
dps_geral['dps'] = dps_geral['damage'] / (dps_geral['time'] - dps_geral['startTime'])

# Calcular média do DPS para cada pokemon
dps_media = dps_geral.groupby(['nome', 'pokemon'])['dps'].mean().reset_index()


## DPS de cada suporte

In [387]:
# Filtrar registros com pokemon diferente de -1
dps_media_suporte = dps_media.query('pokemon != -1')

# Criar gráfico interativo com Plotly Express
fig = px.bar(
    dps_media_suporte,
    x='nome',
    y='dps',
    labels={'avg_dps': 'DPS Médio', 'pokemon_name': 'Suporte'},
)

# Ajustar layout para melhor visualização
#fig.update_layout(xaxis_tickangle=-45)

#Exibir o gráfico
#fig.show()

## Dps Heroi

In [388]:
stats_df = dps_geral.copy()

dps_media_heroi = stats_df.query('pokemon == -1').reset_index(drop=True)

# Adicionar coluna 'hero' com os IDs dos heróis repetidos
dps_media_heroi['hero'] = info_df['character'].repeat(info_df['statistics'].apply(len).tolist()).reset_index(drop=True)

# Adicionar coluna 'hero_nome' com os nomes correspondentes aos heróis
dps_media_heroi['hero_nome'] = info_df['character'].apply(hero_nome_id)

# Calcular a média do DPS para cada herói
dps_media_heroi = dps_media_heroi.groupby('hero_nome')['dps'].mean().reset_index()

# Renomear a coluna do índice para 'hero_nome'
dps_media_heroi = dps_media_heroi.rename(columns={'hero_nome': 'hero_nome'})

# Exibir o DataFrame resultante
dps_media_heroi

,hero_nome,dps
0,Bulbasaur,194.578523
1,Charmander,148.604739
2,Pikachu,191.911764
3,Squirtle,149.689138


In [385]:
info_df_id_partida = info_df.copy()

# Adicionar uma nova coluna 'id_partida' que representa o ID único para cada linha
info_df_id_partida['id_partida'] = info_df_id_partida.index + 1 

# Filtrar linhas da coluna 'pokemon'
stats_df_id_partidah = stats_df.query('pokemon == -1').reset_index(drop=True)

# Adicionar coluna 'hero' e 'id_partida' com os IDs dos heróis repetidos
stats_df_id_partidah['hero'] = info_df_id_partida['character'].reset_index(drop=True)
stats_df_id_partidah['id_partida'] = info_df_id_partida['id_partida'].reset_index(drop=True)

# Adicionar coluna 'hero_nome' com os nomes correspondentes aos heróis
stats_df_id_partidah['nome'] = stats_df_id_partidah['hero'].apply(hero_nome_id)


stats_df_id_partidas = stats_df.copy()

stats_df_id_partidas['id_partida'] = info_df_id_partida['id_partida'].repeat(info_df_id_partida['statistics'].apply(len).tolist()).reset_index(drop=True)

# Filtrar linhas da coluna 'pokemon'
stats_df_id_partidas = stats_df_id_partidas.query('pokemon != -1').reset_index(drop=True)

stats_df_id_partidas



,startTime,pokemon,damage,name,nome,time,dps,id_partida
0,122.7551,3,12869,Xanad0,Kingler,1280.947998,11.111275,1
1,217.8720,0,21683,Xanad0,Arcanine,1280.947998,20.396472,1
2,435.9753,19,24401,Xanad0,Umbreon,1280.947998,28.877856,1
3,806.3627,6,16531,Xanad0,Beedrill,1280.947998,34.832516,1
4,991.2443,8,9001,Xanad0,Electabuzz,1280.947998,31.069676,1
...,...,...,...,...,...,...,...,...
820,221.9673,4,43221,Mestre LpK,Leafeon,3599.999002,12.794729,165
821,690.0281,16,77111,Mestre LpK,Alakazam,3599.999002,26.498890,165
822,1501.2180,11,75893,Mestre LpK,Jolteon,3599.999002,36.160514,165
823,3171.3830,13,0,Mestre LpK,MrMime,3599.999002,0.000000,165
